In [25]:
# load libraries
import pandas as pd
import openpyxl

In [26]:
# import election results for state senate district 9
district9_results = pd.read_excel('detail.xlsx', sheet_name='2', skiprows=1)

/Users/amahaja/GitHub/rehmet_election/.venv/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [27]:
district9_results.head()

,Unnamed: 0,Unnamed: 1,Taylor Rehmet,Unnamed: 3,Unnamed: 4,Unnamed: 5,Leigh Wambsganss,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Precinct,Registered Voters,Absentee,Early,Election,Total Votes,Absentee,Early,Election,Total Votes,Total
1,1001,1940,3,99,98,200,1,43,45,89,289
2,1005,942,3,21,51,75,0,1,2,3,78
3,1008,314,1,12,24,37,0,0,6,6,43
4,1010,698,1,0,20,21,0,2,0,2,23


In [28]:
# there's a total row we don't want at the end, so let's drop that
district9_results.tail()

,Unnamed: 0,Unnamed: 1,Taylor Rehmet,Unnamed: 3,Unnamed: 4,Unnamed: 5,Leigh Wambsganss,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
377,4836,3126,10,76,123,209,4,67,56,127,336
378,4844,1820,2,40,73,115,0,29,44,73,188
379,4846,1720,3,49,39,91,2,16,33,51,142
380,4849,3336,1,83,137,221,0,39,81,120,341
381,Total:,617753,1115,24571,28594,54280,608,19375,20617,40600,94880


In [29]:
# drop last row
district9_results = district9_results.drop(381).reset_index(drop=True)

In [30]:
# how many rows in the dataframe?
len(district9_results)

381

In [31]:
# rename columns to combine candidate names with vote types -- essentially flatten since there are multiple levels of headers
district9_results.columns = [
    'Precinct',
    'Registered_Voters',
    'absentee_TaylorRehmet',
    'early_TaylorRehmet',
    'election_TaylorRehmet',
    'totalVotes_TaylorRehmet',
    'absentee_LeighWambsganss',
    'early_LeighWambsganss',
    'election_LeighWambsganss',
    'totalVotes_LeighWambsganss',
    'Total'
]

# drop first row so we only have one row of headers
district9_results = district9_results.drop(0).reset_index(drop=True)

district9_results.head()

,Precinct,Registered_Voters,absentee_TaylorRehmet,early_TaylorRehmet,election_TaylorRehmet,totalVotes_TaylorRehmet,absentee_LeighWambsganss,early_LeighWambsganss,election_LeighWambsganss,totalVotes_LeighWambsganss,Total
0,1001,1940,3,99,98,200,1,43,45,89,289
1,1005,942,3,21,51,75,0,1,2,3,78
2,1008,314,1,12,24,37,0,0,6,6,43
3,1010,698,1,0,20,21,0,2,0,2,23
4,1011,1134,3,19,45,67,0,2,3,5,72


In [32]:
# verify types
district9_results.dtypes

Precinct                         str
Registered_Voters             object
absentee_TaylorRehmet         object
early_TaylorRehmet            object
election_TaylorRehmet         object
totalVotes_TaylorRehmet       object
absentee_LeighWambsganss      object
early_LeighWambsganss         object
election_LeighWambsganss      object
totalVotes_LeighWambsganss    object
Total                         object
dtype: object

In [33]:
# read in vtd census demographics data
vtd_census_2020 = pd.read_csv('vtd_census - VTDs_24PG_Pop.csv')

In [34]:
# this has demographics for all of the precincts texas
len(vtd_census_2020)

9712

In [35]:
vtd_census_2020.dtypes

CountyFIPS    int64
County          str
CNTY          int64
VTD             str
CNTYVTD         str
VTDKEY        int64
total         int64
anglo         int64
nanglo        int64
asian         int64
black         int64
hisp          int64
bh            int64
vap           int64
anglovap      int64
nanglovap     int64
asianvap      int64
blackvap      int64
hispvap       int64
bhvap         int64
dtype: object

In [36]:
# ensure type is string
vtd_census_2020['VTD'] = vtd_census_2020['VTD'].astype(str)


In [37]:
# we're lucky that all of district 9 is in tarrant county, so can filter to just that
vtd_census_tarrant = vtd_census_2020[vtd_census_2020['County'] == 'TARRANT']

I looked at this data in qgis as well, and saw that six precincts were missing from the VTD data. 

Four were missing because in the VTD data, they had suffixes like A or B. These were missing initially:
- 4737 is split into 4737A and 4737B
- 4846 is split into 4846A and 4846B
- 4245 is split into 4245A and 4245B
- 3325 is split into 3325A and 3325B

The following were also missing from the VTD data, but these were gone entirely:
- Precinct 1823 NOT found in vtd_census_2020 
- Precinct 4598 NOT found in vtd_census_2020

Looking back at the election results data, each of these precincts had zero votes.

In [38]:
# let's aggregate the demographics across precincts with A and B suffixes

# create base column that strips the A/B suffix
vtd_census_tarrant['VTD_base'] = vtd_census_tarrant['VTD'].str.rstrip('AB')

# identify which VTDs we are adding combined
combined_vtds = vtd_census_tarrant.groupby('VTD_base').filter(lambda x: len(x) > 1)
if len(combined_vtds) > 0:
    print("VTDs that will be combined:")
    for base_vtd in combined_vtds['VTD_base'].unique():
        variants = vtd_census_tarrant[vtd_census_tarrant['VTD_base'] == base_vtd]['VTD'].tolist()
        print(f"  {base_vtd}: {', '.join(variants)}")
    print()

# specify which columns to sum when combining A and B precincts
cols_to_add = ['total',
               'anglo',
               'nanglo',
               'asian',
               'black',
               'hisp',
               'bh',
               'vap',
               'anglovap',
               'nanglovap',
               'asianvap',
               'blackvap',
               'hispvap',
               'bhvap']

# we want to group by the stripped VTD name and sum specified columns, keep first value for others
agg_dict = {col: 'sum' if col in cols_to_add else 'first' 
            for col in vtd_census_tarrant.columns if col not in ['VTD', 'VTD_base']}

vtd_census_combined = vtd_census_tarrant.groupby('VTD_base', as_index=False).agg(agg_dict)

# rename VTD_base back to VTD
vtd_census_combined.rename(columns={'VTD_base': 'VTD'}, inplace=True)

print(f"Original Tarrant VTDs: {len(vtd_census_tarrant)}")
print(f"Combined VTDs: {len(vtd_census_combined)}")

VTDs that will be combined:
  2827: 2827A, 2827B
  3325: 3325A, 3325B
  4245: 4245A, 4245B
  4737: 4737A, 4737B
  4846: 4846A, 4846B

Original Tarrant VTDs: 782
Combined VTDs: 777


In [39]:
# join both data frames on Precinct and VTD
# the key to join is NOT vtdkey -- values for that are as low as 1. there's no precinct 1 in district 9

merged_df = pd.merge(district9_results, vtd_census_combined, left_on='Precinct', right_on='VTD', how='inner')

In [41]:
# are there any duplicate precincts now?
merged_df.duplicated(subset=['Precinct']).sum()

np.int64(0)

In [42]:
# how many precincts are missing?
len(merged_df)

378

In [43]:
# show precincts that are in in district9_results but not in merged_df
precincts_in_original = set(district9_results['Precinct'])
precincts_in_merged = set(merged_df['Precinct'])
missing_precincts = precincts_in_original - precincts_in_merged

print(f"Precincts not in merged: {len(missing_precincts)}")
if missing_precincts:
    print("Missing precincts:")
    for p in missing_precincts:
        print(f"{p}")

Precincts not in merged: 2
Missing precincts:
4598
1823


In [44]:
# check if these missing precincts are in original vtd_census_2020 data
for precinct in missing_precincts:
    match = vtd_census_2020[vtd_census_2020['VTD'] == precinct]
    if len(match) > 0:
        print(f"Precinct {precinct} found in vtd_census_2020:")
        print(f"  Counties: {match['County'].unique().tolist()}")
    else:
        print(f"Precinct {precinct} not in vtd_census_2020")

Precinct 4598 not in vtd_census_2020
Precinct 1823 not in vtd_census_2020


In [45]:
# see these rows in the election results data 
# each of them have zero total votes
district9_results[district9_results['Precinct'].isin(['1823', '4598'])]

,Precinct,Registered_Voters,absentee_TaylorRehmet,early_TaylorRehmet,election_TaylorRehmet,totalVotes_TaylorRehmet,absentee_LeighWambsganss,early_LeighWambsganss,election_LeighWambsganss,totalVotes_LeighWambsganss,Total
75,1823,0,0,0,0,0,0,0,0,0,0
339,4598,0,0,0,0,0,0,0,0,0,0


In [46]:
# were there other precincts with zero total votes?
# this is weird because there are others with no votes but they still show up
district9_results[district9_results['Total'] == 0]

,Precinct,Registered_Voters,absentee_TaylorRehmet,early_TaylorRehmet,election_TaylorRehmet,totalVotes_TaylorRehmet,absentee_LeighWambsganss,early_LeighWambsganss,election_LeighWambsganss,totalVotes_LeighWambsganss,Total
11,1064,0,0,0,0,0,0,0,0,0,0
38,1420,0,0,0,0,0,0,0,0,0,0
40,1434,0,0,0,0,0,0,0,0,0,0
42,1455,0,0,0,0,0,0,0,0,0,0
43,1457,2,0,0,0,0,0,0,0,0,0
49,1491,5,0,0,0,0,0,0,0,0,0
50,1544,11,0,0,0,0,0,0,0,0,0
51,1547,0,0,0,0,0,0,0,0,0,0
52,1549,0,0,0,0,0,0,0,0,0,0
54,1586,11,0,0,0,0,0,0,0,0,0


In [47]:
# check for nas
merged_df.isna().sum()

Precinct                      0
Registered_Voters             0
absentee_TaylorRehmet         0
early_TaylorRehmet            0
election_TaylorRehmet         0
totalVotes_TaylorRehmet       0
absentee_LeighWambsganss      0
early_LeighWambsganss         0
election_LeighWambsganss      0
totalVotes_LeighWambsganss    0
Total                         0
VTD                           0
CountyFIPS                    0
County                        0
CNTY                          0
CNTYVTD                       0
VTDKEY                        0
total                         0
anglo                         0
nanglo                        0
asian                         0
black                         0
hisp                          0
bh                            0
vap                           0
anglovap                      0
nanglovap                     0
asianvap                      0
blackvap                      0
hispvap                       0
bhvap                         0
dtype: i

In [48]:
# export csv
merged_df.to_csv('district9_election_census_2020.csv', index=False)